In [24]:
import rasterio
import traceback
import numpy as np

from rasterio.transform import from_origin
from esa_snappy import ProductIO, jpy

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

In [35]:
def get_gc_and_write_tif(path_to_sentinel_data, output_tif_path):
    # Read the Sentinel product
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    
    # Get the GeoCoding information
    gc = product_subset.getSceneGeoCoding()
    
    # Get the band names
    bands_names = list(product_subset.getBandNames())
    
    # Access the first band (or another band of interest)
    band = product_subset.getBand(bands_names[0])
    
    # Read the raster data of the band
    width = band.getRasterWidth()
    height = band.getRasterHeight()
    
    # Create an array to store the pixel data
    pixels = np.zeros((width * height,), dtype=np.float32)
    
    # Read pixel data into the array
    band.readPixels(0, 0, width, height, pixels)
    
    # Reshape the pixel data to match the width and height
    raster_data = pixels.reshape((height, width))
    
    # Calculate the pixel size by comparing geolocation of two adjacent pixels
    pixel_pos_00 = jpy.get_type('org.esa.snap.core.datamodel.PixelPos')(0, 0)
    pixel_pos_10 = jpy.get_type('org.esa.snap.core.datamodel.PixelPos')(1, 0)
    pixel_pos_01 = jpy.get_type('org.esa.snap.core.datamodel.PixelPos')(0, 1)
    
    geo_pos_00 = gc.getGeoPos(pixel_pos_00, None)
    geo_pos_10 = gc.getGeoPos(pixel_pos_10, None)
    geo_pos_01 = gc.getGeoPos(pixel_pos_01, None)
    
    pixel_size_x = geo_pos_10.getLon() - geo_pos_00.getLon()
    pixel_size_y = geo_pos_01.getLat() - geo_pos_00.getLat()
    
    # Retrieve the upper left corner coordinates
    lon = geo_pos_00.getLon()
    lat = geo_pos_00.getLat()
    
    # Construct the affine transformation
    transform = from_origin(lon, lat, pixel_size_x, -pixel_size_y)  # Negative for north-up images

    # Get the CRS in WKT format
    # crs_wkt = gc.getProjection().toWKT()
    
    # Define the GeoTIFF profile using rasterio
    profile = {
        'driver': 'GTiff',
        'dtype': raster_data.dtype,
        'nodata': None,
        'width': width,
        'height': height,
        'count': 1,  # Number of bands
        # 'crs': crs_wkt,  # Set the CRS
        'transform': transform  # Set the affine transform
    }
    
    # Write the data to a GeoTIFF file
    with rasterio.open(output_tif_path, 'w', **profile) as dst:
        dst.write(raster_data, 1)  # Write data to the first band
    
    return product_subset, gc, bands_names


In [36]:
# Example usage
path_to_sentinel_data = '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MAN/20230501_20230512.dim'
output_tif_path = 'your_file.tif'
product, geo_coding, band_names = get_gc_and_write_tif(path_to_sentinel_data, output_tif_path)

